From [https://www.kaggle.com/c/coleridgeinitiative-show-us-the-data]

# Early Load of Functions

## walk_directory(directory)

In [1]:
def walk_directory(directory):
    import os
    
    iterator = 0
    file_list = []

    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.startswith('.'):
                pass
            else:
                pathway = os.path.join(root, file)
                pathway = os.path.realpath(pathway)# added from Jeanna for Windows to read
                file_list.append(pathway)
                iterator += 1
                
    return file_list

## determine_os( )

In [2]:
def determine_os():
    import platform
    system = platform.system()
    if system == 'Darwin':
        os_system = 'Mac'
    elif system == 'Windows':
        os_system = 'Windows'
    return os_system

## walk_mac_directory( )

In [3]:
def walk_mac_directory_json(directory):
    import os
    iterator = 0
    file_list = []

    for root, dirs, files in tqdm(os.walk(directory)):
        for file in files:
            if file.startswith('.'):
                pass
            elif file.endswith('json'):
                pathway = os.path.join(root, file)
                file_list.append(pathway)
            else:
                pass

                
    return file_list


## walk_pc_directory( )

In [4]:
def walk_pc_directory_json(directory):
    import os
    
    file_list = []

    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.startswith('.'):
                pass
            elif file.endswith('json'):
                pathway = os.path.realpath(pathway)# added from Jeanna for Windows to read
                file_list.append(pathway)
            else:
                pass
                
                
    return file_list

# Step 1: Looking at the Sample Output DataFrame

Kaggle provided a sample output DataFrame. We are examining it to see what is in it. 

In [9]:
import pandas as pd

sample_df = pd.read_csv('coleridgeinitiative-show-us-the-data/sample_submission.csv')

print(sample_df.shape)

display(sample_df)

(4, 2)


,Id,PredictionString
0,2100032a-7c33-4bff-97ef-690822c43466,NaN
1,2f392438-e215-4169-bebf-21ac4ff253e1,NaN
2,3f316b38-1a24-45a9-8d8c-4e05a42257c6,NaN
3,8e6996b4-ca08-4c0b-bed2-aaf07a4c6a60,NaN


Annnnnddddd... looking at the sample submission DataFrame, it does not tell us anything. 

# Step 2: Looking at the train.csv DataFrame

Kaggle also included a train.csv file. Let's look at that file to see what is in it. 

In [13]:
import pandas as pd

train_df = pd.read_csv('coleridgeinitiative-show-us-the-data/train.csv')

print(train_df.shape)

display(train_df.head(3).T)

(19661, 5)


,0,1,2
Id,d0fa7568-7d8e-4db9-870f-f9c6f668c17b,2f26f645-3dec-485d-b68d-f013c9e05e60,c5d5cd2c-59de-4f29-bbb1-6a88c7b52f29
pub_title,The Impact of Dual Enrollment on College Degre...,Educational Attainment of High School Dropouts...,Differences in Outcomes for Female and Male St...
dataset_title,National Education Longitudinal Study,National Education Longitudinal Study,National Education Longitudinal Study
dataset_label,National Education Longitudinal Study,National Education Longitudinal Study,National Education Longitudinal Study
cleaned_label,national education longitudinal study,national education longitudinal study,national education longitudinal study


And again ... this does not show us much. 

Kaggle did include a folder (called "train") that contains json files. These json files are of articles. This will be our next examination.

# Step 3: Examine one of the json files in the train folder

In [14]:
import pandas as pd

example_json = pd.read_json('coleridgeinitiative-show-us-the-data/train/0007f880-0a9b-492d-9a58-76eb0b0e0bd7.json')

example_json

,section_title,text
0,Abstract,The aim of this study was to identify if acqui...
1,Introduction,The spur of ICT (Information and Communication...
2,English and ICT,Studies continue to show that primary language...
3,Lebanon,Lebanon is a small country situated in the Mid...
4,Gender Inequality in Lebanon,Lebanese women are in a constant battle with a...
5,Lebanon's ICT Sector,While a number of Arab governments have adopte...
6,DOT Lebanon ICT Training Program,DOT Lebanon is a registered local Lebanese NGO...
7,Methodology,
8,Study Design,This is a prospective cohort study describing ...
9,Study Procedure,"Upon registering in the training, participants..."


So the json files are broken down into each sections based upon the section title in each article. This will mean that if we build a single csv/DataFrame of all of the articles in the folder, we will need to add an additional column to the final csv/DataFrame that gives us the title of the article. The DataFrame columns would be **['article_title', 'section_title', 'text']**. This will give us a way to refer back to each article individually, as well as into each section of each article specifically.

# Step 4: How many files are in the train folder?

Earlier (section 1.1 above) we established a function to walk a directory. We are going to use this function next to walk the train folder and determine
1. how many files are in the folder, and
1. what the titles of the json files.

In [27]:
import pandas as pd
import os
from tqdm.auto import trange#, tqdm_notebook

directory = 'coleridgeinitiative-show-us-the-data/train'

os_system = determine_os()

if os_system == 'Mac':
    file_list = walk_mac_directory_json(directory)
elif os_system == 'Windows':
    file_list = walk_pc_directory_json(directory)
else:
    print("Error in reading the files.")

print(len(file_list))

1it [00:00, 20.06it/s]

14316


# Step 5: Do we want to combine all of the text into one csv?

I am not sure if we want to do this, but this section will combine everything into one **HUGE** csv. 

In [33]:
df = pd.DataFrame(columns= ['title', 'text'])
df.text = ''



for iterator, element in tqdm(enumerate(file_list)):
    df.loc[iterator, 'title'] = element
    temp_df = pd.read_json(element)
    for text in temp_df['text']:
        df.loc[iterator, 'text'] = str(df.loc[iterator, 'text']) + str(text)

    
len(df)

14316it [02:37, 91.05it/s] 


14316

In [34]:
df.head()

,title,text
0,coleridgeinitiative-show-us-the-data/train/021...,"nanOn 26 January 2010, at an ordinary airport ..."
1,coleridgeinitiative-show-us-the-data/train/569...,nanAbstract This study examined disparities in...
2,coleridgeinitiative-show-us-the-data/train/736...,nanMany small American farms struggle to remai...
3,coleridgeinitiative-show-us-the-data/train/21e...,nanAbstract-We describe a method for automated...
4,coleridgeinitiative-show-us-the-data/train/4fc...,"nanIn the U.S. and across many countries, busi..."


In [35]:
df.to_csv('all_json_files_combined.csv')